In [1]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from pygame import mixer

pygame 2.1.2 (SDL 2.0.18, Python 3.8.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


I used haar cascade to detect the face and eyes.

In [2]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +
                                     'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +
                                    'haarcascade_eye.xml')

I used the model weights  from the second part of the project.

In [3]:
model = load_model(r'/Users/amrutesh/Downloads/MRL-Eyes/CVmodel2.h5')

2022-06-03 09:16:09.054334: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
mixer.init()

I loaded an audio file in mixer library. This alarm will be raised if a person is drowsy.

In [5]:
sound = mixer.Sound(r'/Users/amrutesh/Downloads/MRL-Eyes/alarm.wav')

In [6]:
cap = cv2.VideoCapture(0)
Score = 0
while True:
    ret, frame = cap.read()
    height, width = frame.shape[0:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray,
                                          scaleFactor=1.2,
                                          minNeighbors=3)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=1)

    cv2.rectangle(frame, (0, height - 50), (200, height), (0, 0, 0),
                  thickness=cv2.FILLED)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame,
                      pt1=(x, y),
                      pt2=(x + w, y + h),
                      color=(255, 0, 0),
                      thickness=3)

    for (ex, ey, ew, eh) in eyes:
        cv2.rectangle(frame,
                      pt1=(ex, ey),
                      pt2=(ex + ew, ey + eh),
                      color=(0, 76, 0),
                      thickness=3)

        # preprocessing steps
        eye = frame[ey:ey + eh, ex:ex + ew]
        eye = cv2.resize(eye, (80, 80))
        eye = eye / 255
        eye = eye.reshape(80, 80, 3)
        eye = np.expand_dims(eye, axis=0)
        # preprocessing is done now model prediction
        prediction = model.predict(eye)

        # if eyes are closed
        if prediction[0][0] > 0.90:
            cv2.putText(frame,
                        'closed', (10, height - 20),
                        fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,
                        fontScale=1,
                        color=(255, 255, 255),
                        thickness=2,
                        lineType=cv2.LINE_AA)
            cv2.putText(frame,
                        'Score' + str(Score), (100, height - 20),
                        fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,
                        fontScale=1,
                        color=(255, 255, 255),
                        thickness=2,
                        lineType=cv2.LINE_AA)
            Score = Score + 1
            if (Score > 15):
                try:
                    sound.play()
                except:
                    pass

        # if eyes are open
        elif prediction[0][1] > 0.90:
            cv2.putText(frame,
                        'open', (10, height - 20),
                        fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,
                        fontScale=1,
                        color=(255, 255, 255),
                        thickness=1,
                        lineType=cv2.LINE_AA)
            cv2.putText(frame,
                        'Score' + str(Score), (100, height - 20),
                        fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,
                        fontScale=1,
                        color=(255, 255, 255),
                        thickness=1,
                        lineType=cv2.LINE_AA)
            Score = Score - 1
            if (Score < 0):
                Score = 0

    cv2.imshow('frame', frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

2022-06-03 09:16:14.819000: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
